In [17]:
# pip install giotto-tda
# pip install gudhi

In [35]:
import pandas as pd
import numpy as np
from gtda.time_series import SingleTakensEmbedding, TakensEmbedding
import gudhi 
import math
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math

# importing Qiskit
from qiskit import transpile, assemble
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
import qiskit.quantum_info as qi


# import basic plot tools
from qiskit.visualization import plot_histogram

time_series = np.log(pd.read_csv("SP500.csv", header=None).to_numpy().squeeze())

In [19]:
# initial values for important variables -- will have to find optimal values later on

n = 4 # dim of vectors
d = 5 # time delay
w = 5 # window size
epsilon = 0.03 # resolution threshold
q = 3 # number of precision qubits
simplex_dim = 4 # maximum simplex tree dimension used

np.set_printoptions(precision=3, suppress=True)

In [20]:
def TakenEmbedding(time_series, dim, delay):
    serie = time_series.reshape(1, -1)
    TE = TakensEmbedding(time_delay=delay, dimension=dim)
    return TE.fit_transform(serie)[0]

In [21]:
def TakenPointCloud(time_series, dim, delay, window_size):
    embedding = TakenEmbedding(time_series, dim, delay)
    point_cloud = []
    l = len(embedding)
    for i in range(l - window_size):
        window = embedding[i : i + window_size]
        point_cloud.append(window)

    return np.array(point_cloud)

In [22]:
def SimplexTrees(clouds, simplices, resolution):
    tree = []

    for i in range(len(clouds)):
        rips_complex = gudhi.RipsComplex(points = clouds[i], max_edge_length = resolution)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension = simplices)

        simplex = [val for val, dist in simplex_tree.get_filtration()]

        simplex = sorted(simplex, key=lambda vector: (len(vector), vector))

        tree.append(simplex)

    return tree

In [23]:
import scipy

def unitarize(h):
    if not np.allclose(h, h.conj().T, atol = 1e-8):
        raise ValueError("Input matrix H must be Hermitian.")
    ih = 1j * h
    return scipy.linalg.expm(ih)

In [24]:
def BoundaryOperators(simplex_tree, dimension):
    minus = [elem for elem in simplex_tree if len(elem) == dimension] # for 1-simplicities will find all points, etc.
    forms = [elem for elem in simplex_tree if len(elem) == dimension + 1] # for 1-simplicities will find all edges, etc. 

    if(dimension == 0):
        return np.array([[(-1) ** i] for i in range(len([item for item in simplex_tree if len(item) == 1]))])

    operator = []

    for simplex_minus in minus:
        simplices = []

        for simplex_form in forms:
            if(all(item in simplex_form for item in simplex_minus) and len(simplex_minus) + 1 == len(simplex_form)):
                val = list(set(simplex_form).symmetric_difference(simplex_minus))[0]
                if(val % 2 == 1):
                    simplices.append(-1)
                else:
                    simplices.append(1)
            else:
                simplices.append(0)

        operator.append(simplices)

    return np.array(operator)

In [25]:
def CombinatorialLaplacian(tree, index):    
    delta_k = np.array(BoundaryOperators(tree, index))
    delta_l = np.array(BoundaryOperators(tree, index + 1))

    temp = np.matmul(delta_k.conj().T, delta_k) + np.matmul(delta_l, delta_l.conj().T)

    points = len([items for items in tree if len(items) == index + 1])
    dim = 2 ** math.ceil(math.log(points, 2))
    pad = dim - points
    temp = np.pad(temp, (0, pad))

    gershgorin = []
    for i in range(len(temp)):
        sum = 0
        
        for j in range(len(temp[i])):
            if(i != j):
                sum += np.abs(temp[i][j])
        
        gershgorin.append(sum)

    max_eigenval = max(gershgorin)
    
    for i in range(points, dim):
        temp[i][i] = max_eigenval / 2
    
    return temp * 6 / max_eigenval # return val should be Hermitian

In [32]:
point_clouds = TakenPointCloud(time_series, n, d, w)
# point_cloud[t] gives the point cloud at time t

In [53]:
simplex_trees = SimplexTrees(point_clouds, simplex_dim, epsilon)
# simplex_trees[t] gives the simplex trees for the point cloud at time t

63


In [54]:
laplacians = [[unitarize(CombinatorialLaplacian(simplex_trees[i], 0)), unitarize(CombinatorialLaplacian(simplex_trees[i], 0))] for i in range(len(simplex_trees))]
# laplacians[t][i] gives the ith Laplacian matrix of the point cloud at time t

63


In [29]:
def qft_dagger(qc, n):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-math.pi/float(2**(j-m)), m, j)
        qc.h(j)

In [113]:
# Create and set up circuit
counting_q = 3
kickback_q = int(math.log(len(laplacians[0][0]), 2))
qpe_L = QuantumCircuit(kickback_q, 0)
qpe_f = QuantumCircuit(kickback_q + counting_q, 0)

# Apply H-Gates to counting qubits:
for qubit in range(counting_q):
    qpe_f.h(qubit)

unitary_matrix = qi.Operator(laplacians[0][0])

qubit_array = [q for q in range(kickback_q)]

qpe_L.unitary(unitary_matrix, qubit_array, label="L")
L = qpe_L.to_gate().control(1)

for i in range(3):
    for j in range(2 ** i):
        qpe_f.append(L, [i, 3, 4, 5])

qpe_f.draw()


┌───┐                                                                    »
q_0: ┤ H ├────────■───────────────────────────────────────────────────────────»
     ├───┤        │                                                           »
q_1: ┤ H ├────────┼────────────────■────────────────■─────────────────────────»
     ├───┤        │                │                │                         »
q_2: ┤ H ├────────┼────────────────┼────────────────┼────────────────■────────»
     └───┘┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐»
q_3: ─────┤0              ├┤0              ├┤0              ├┤0              ├»
          │               ││               ││               ││               │»
q_4: ─────┤1 circuit-1441 ├┤1 circuit-1441 ├┤1 circuit-1441 ├┤1 circuit-1441 ├»
          │               ││               ││               ││               │»
q_5: ─────┤2              ├┤2              ├┤2              ├┤2              ├»
          └───────────────┘└───────────────┘└───────────────┘└───────────────┘»
«                                                        
«q_0: ───────────────────────────────────────────────────
«                                                        
«q_1: ───────────────────────────────────────────────────
«                                                        
«q_2: ────────■────────────────■────────────────■────────
«     ┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐
«q_3: ┤0              ├┤0              ├┤0              ├
«     │               ││               ││               │
«q_4: ┤1 circuit-1441 ├┤1 circuit-1441 ├┤1 circuit-1441 ├
«     │               ││               ││               │
«q_5: ┤2              ├┤2              ├┤2              ├
«     └───────────────┘└───────────────┘└───────────────┘

In [99]:
qpe3 = QuantumCircuit(kickback_q, 0)
unitary_matrix = qi.Operator(laplacians[0][0])
qubit_array = []
for q in range(kickback_q):
    qubit_array.append(q)

qpe3.unitary(unitary_matrix, qubit_array, label="L")
qpe_controlled = qpe3.control(1)
qpe_controlled.draw()

q8: ───────■────────
     ┌──────┴───────┐
q_0: ┤0             ├
     │              │
q_1: ┤1 circuit-540 ├
     │              │
q_2: ┤2             ├
     └──────────────┘